In [9]:
import pandas as pd 
import numpy as np
import joblib

from sklearn.svm import SVC



In [5]:
clf_cargado = joblib.load('modelos/svm_optimized.joblib')


In [8]:
clf_cargado.get_params()

{'C': 9,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': np.float64(0.0700434954523006),
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

Prueba con registro fijo

In [11]:
registro_prueba = np.array([-0.61324414,  1.79912259, -1.37171857, -0.02432881, -0.96784862,
       -1.09801174, -0.68090848, -0.27080128,  0.79136757, -0.28379026])

pred = clf_cargado.predict([registro_prueba])
print(f'La predicción para el registro de prueba es: {pred[0]}')

La predicción para el registro de prueba es: Medio Riesgo


/home/orlandodua/Documentos/aprendePython/luminaria/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


## Importar Pipeline

In [ ]:
# Como el pipeline fue contruido en base a un DataFrame
# y en base a un BaseEstimator personalizado, es necesario
# replicar esas partes aquí para poder usar el pipeline

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

binarias = ['dificultades_economicas_bin', 'reprobo_materias_bin']
ordinales = {
    'semestre_ord': ['1er - 3er', '4to - 6to', '7mo o más'],
    'apoyo_institucional_ord': ['Nunca', 'Algunas veces', 'Siempre'],
    'satisfaccion_servicios_ord': ['Muy insatisfecho/a', 'Insatisfecho/a', 'Satisfecho/a', 'Muy satisfecho/a'],
    'actividades_extracurriculares_ord': ['Nunca', 'Algunas veces', 'Sí, frecuentemente']
}
onehot = ['empleo_ord', 'impacto_laboral_ord']


class BinarioMapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.maps = {
            'dificultades_economicas_bin': {'Sí': 1, 'No': 0},
            'reprobo_materias_bin': {'Sí': 1, 'No': 0}
        }
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_ = X.copy()
        for col, mapping in self.maps.items():
            X_[col + '_bin'] = X_[col].map(mapping)
        return X_[[col + '_bin' for col in self.maps]]

preprocessor = ColumnTransformer(
    transformers=[
        # Binarias
        ('bin', BinarioMapper(), binarias),
        # Ordinales
        ('ord', OrdinalEncoder(categories=[ordinales[k] for k in ordinales], dtype=int),
         list(ordinales.keys())),
        # One-hot
        ('onehot', OneHotEncoder(drop='first', sparse_output=False, dtype=int), onehot)
    ]
)

In [ ]:
# Con lo anterior, ahora podemos cargar el pipeline completo
pipeline = joblib.load('./modelos/svm_preprocessor.joblib')

In [16]:
df_prueba = pd.read_csv('./datasets/luminaria_con_niveles_de_riesgo 231.csv')
X_prueba = df_prueba.drop('nivel_riesgo', axis=1)
X_prueba.head()

,semestre_ord,desmotivacion_bin,considerado_abandonar_bin,dificultades_economicas_bin,empleo_ord,impacto_laboral_ord,reprobo_materias_bin,apoyo_institucional_ord,satisfaccion_servicios_ord,actividades_extracurriculares_ord
0,7mo o más,No,Sí,No,"Sí, medio tiempo",Algo,No,Algunas veces,Insatisfecho/a,Algunas veces
1,7mo o más,No,Sí,Sí,"Sí, medio tiempo",Algo,No,Siempre,Insatisfecho/a,"Sí, frecuentemente"
2,1er - 3er,Si,Sí,Sí,No,Algo,No,Algunas veces,Insatisfecho/a,Nunca
3,1er - 3er,Si,No,Sí,"Sí, medio tiempo",Algo,No,Algunas veces,Satisfecho/a,Algunas veces
4,7mo o más,No,No,No,"Sí, medio tiempo",No afecta/No trabajo,No,Algunas veces,Satisfecho/a,"Sí, frecuentemente"


In [ ]:
X_prueba.iloc[10]

semestre_ord                                7mo o más
desmotivacion_bin                                  No
considerado_abandonar_bin                          Sí
dificultades_economicas_bin                        No
empleo_ord                           Sí, medio tiempo
impacto_laboral_ord                              Algo
reprobo_materias_bin                               No
apoyo_institucional_ord                 Algunas veces
satisfaccion_servicios_ord             Insatisfecho/a
actividades_extracurriculares_ord       Algunas veces
Name: 0, dtype: object

In [20]:
registro_prueba = pipeline.transform(X_prueba.iloc[[10]])
registro_prueba

array([[ 1.63067192, -0.55582649,  0.90505143, -0.02432881, -0.96784862,
        -1.09801174, -0.68090848, -0.27080128,  0.79136757, -0.28379026]])

In [24]:
pred = clf_cargado.predict(registro_prueba)
valor_real = df_prueba['nivel_riesgo'].iloc[10]

print(f'La predicción para el registro de prueba es: {pred[0]}')
print(f'Valor real: {valor_real}')

La predicción para el registro de prueba es: Medio Riesgo
Valor real: Medio Riesgo


/home/orlandodua/Documentos/aprendePython/luminaria/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
